In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

c:\Users\keqing.li\Documents\pytorch-is-easy\.pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data = data.dropna()

In [3]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data.iloc[idx]["review"], self.data.iloc[idx]["label"]  

In [11]:
from torch.utils.data import random_split
trainset, validset = random_split(dataset, lengths=[0.9, 0.1])


In [ ]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

In [17]:
import torch 

tokenizer = AutoTokenizer.from_pretrained("./rbt3")

def collate_fn(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation = True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)

    return inputs
    

In [18]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size = 32, shuffle=True, collate_fn = collate_fn)
validloader = DataLoader(validset, batch_size = 64, shuffle=True, collate_fn = collate_fn)


In [ ]:
next(enumerate(validloader))[1]

In [ ]:
import torch
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("./rbt3")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if torch.cuda.is_available():
#     model = model.cuda()
model.to(device)

optimizer = Adam(model.parameters(), lr=2e-5)

In [33]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim = -1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validloader.dataset)

def train(epoch=3, log_step=100):
    global_step = 0
    for epoch in range(epoch):
        model.train()
        for step, batch in enumerate(trainloader):
            # 1. Fixed cuda check
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            
            # 2. Clear previous gradients
            optimizer.zero_grad()
            
            output = model(**batch)
            loss = output.loss
            loss.backward()
            
            # 3. Update weights (MISSING IN ORIGINAL)
            optimizer.step()
            
            if global_step % log_step == 0:
                print("global step: {}, loss: {}".format(global_step, loss.item())) 
            global_step += 1
        
        acc = evaluate()
        print("epoch: {}, acc: {}".format(epoch, acc))

In [34]:
train()

global step: 0, loss: 0.2685430645942688
global step: 100, loss: 0.12564055621623993
global step: 200, loss: 0.16682714223861694
epoch: 0, acc: 0.8981958627700806
global step: 300, loss: 0.1955653876066208
global step: 400, loss: 0.14544838666915894
epoch: 1, acc: 0.8969072103500366
global step: 500, loss: 0.12719734013080597
global step: 600, loss: 0.12414306402206421
epoch: 2, acc: 0.9007731676101685
